<a href="https://colab.research.google.com/github/ABDULHAMEED-10/EmotionDetectionModel/blob/main/EmotionAIFaceDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import zipfile
import os

In [29]:
zip_path = '/content/drive/MyDrive/data.zip'  # Path to your uploaded zip file
extract_path = '/content/dataset'  # Destination folder for extraction

In [30]:
# Create a folder to extract the contents of the zip file
os.makedirs(extract_path, exist_ok=True)

In [31]:
# Extract the contents of the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [32]:
import cv2
import tensorflow as tf
import tensorflow
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.models import model_from_json
from google.colab.patches import cv2_imshow


In [33]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [34]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        '/content/dataset/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [35]:
# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        '/content/dataset/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [36]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=tensorflow.keras.optimizers.legacy.Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])

In [37]:
# Train the neural network/model
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/50
448/448 [==============================] - 17s 36ms/step - loss: 1.8009 - accuracy: 0.2592 - val_loss: 1.7159 - val_accuracy: 0.3396
Epoch 2/50
448/448 [==============================] - 15s 33ms/step - loss: 1.6299 - accuracy: 0.3651 - val_loss: 1.5485 - val_accuracy: 0.4106
Epoch 3/50
448/448 [==============================] - 15s 33ms/step - loss: 1.5362 - accuracy: 0.4099 - val_loss: 1.4807 - val_accuracy: 0.4268
Epoch 4/50
448/448 [==============================] - 15s 33ms/step - loss: 1.4627 - accuracy: 0.4374 - val_loss: 1.4107 - val_accuracy: 0.4583
Epoch 5/50
448/448 [==============================] - 15s 34ms/step - loss: 1.4000 - accuracy: 0.4674 - val_loss: 1.3484 - val_accuracy: 0.4900
Epoch 6/50
448/448 [==============================] - 15s 33ms/step - loss: 1.3493 - accuracy: 0.4882 - val_loss: 1.3205 - val_accuracy: 0.5022
Epoch 7/50
448/448 [==============================] - 15s 33ms/step - loss: 1.3040 - accuracy: 0.5029 - val_loss: 1.2770 - val_accuracy:

In [38]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("/content/emotion_model.json", "w") as json_file:
    json_file.write(model_json)

In [39]:
# save trained model weight in .h5 file
emotion_model.save_weights('/content/emotion_model.h5')

In [40]:
# test code start from below for testing run only below files and library files

In [41]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [42]:
# load json and create model
json_file = open('/content/emotion_model.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

In [43]:
# load weights into new model
emotion_model.load_weights("/content/emotion_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [44]:
# Open the input video
input_path = "/content/drive/MyDrive/Video/EMOO.mp4"
cap = cv2.VideoCapture(input_path)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object to save the output video
output_path = '/content/drive/MyDrive/Video/EMOO_with_predictions.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Haar cascade for face detection
face_detector = cv2.CascadeClassifier('/content/drive/MyDrive/haarcascades/haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame = cv2.resize(frame, (1280, 720))
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # Process each detected face
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y - 50), (x + w, y + h + 10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # Predict emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x + 5, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Write the frame with annotations to the output video
    out.write(frame)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer objects
cap.release()
out.release()

# Destroy any OpenCV windows
cv2.destroyAllWindows()


1/1 [==============================] - 0s 17ms/step
